In [2]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output

In [3]:
expiry_sp = 'BANKNIFTY23518'
inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains(expiry_sp))]
# expiry_march = expiry_march.query("(strike >= 38500 )&( strike <= 40500)")
expiry_march['strike_type'] = expiry_march['strike'].astype(int).astype(str) + expiry_march['instrument_type']

In [4]:
# expiry_march  = inst[(inst.name == 'BANKNIFTY') & (inst.tradingsymbol.str.contains('BANKNIFTY234'))].tradingsymbol.str.split('BANKNIFTY').str[1].str[:5].unique()
expiry_march

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,strike_type
2179,11706882,45730,BANKNIFTY2351836000CE,BANKNIFTY,0.0,2023-05-18,36000.0,0.05,25,CE,NFO-OPT,NFO,36000CE
2180,11707138,45731,BANKNIFTY2351836000PE,BANKNIFTY,0.0,2023-05-18,36000.0,0.05,25,PE,NFO-OPT,NFO,36000PE
2181,11707394,45732,BANKNIFTY2351836500CE,BANKNIFTY,0.0,2023-05-18,36500.0,0.05,25,CE,NFO-OPT,NFO,36500CE
2182,11707650,45733,BANKNIFTY2351836500PE,BANKNIFTY,0.0,2023-05-18,36500.0,0.05,25,PE,NFO-OPT,NFO,36500PE
2183,11707906,45734,BANKNIFTY2351837000CE,BANKNIFTY,0.0,2023-05-18,37000.0,0.05,25,CE,NFO-OPT,NFO,37000CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,11774210,45993,BANKNIFTY2351849000PE,BANKNIFTY,0.0,2023-05-18,49000.0,0.05,25,PE,NFO-OPT,NFO,49000PE
2347,11774466,45994,BANKNIFTY2351849500CE,BANKNIFTY,0.0,2023-05-18,49500.0,0.05,25,CE,NFO-OPT,NFO,49500CE
2348,11774722,45995,BANKNIFTY2351849500PE,BANKNIFTY,0.0,2023-05-18,49500.0,0.05,25,PE,NFO-OPT,NFO,49500PE
2349,11774978,45996,BANKNIFTY2351850000CE,BANKNIFTY,0.0,2023-05-18,50000.0,0.05,25,CE,NFO-OPT,NFO,50000CE


In [8]:
start_dt = '2023-04-01'
end_dt = '2023-05-30'
time_frame = '5minute'

df_expiry_hist_data = pd.DataFrame()
strike_symbol_dict = dict(zip(expiry_march.instrument_token,expiry_march['strike_type'] ))
inst_expiry = expiry_march['instrument_token'].unique().tolist() 
for i in inst_expiry:
    df = pd.DataFrame(kite.historical_data(i, from_date = start_dt, to_date = end_dt, interval = time_frame, continuous=False, oi=True))
    df['strike_type'] = strike_symbol_dict[i]
    df_expiry_hist_data = df_expiry_hist_data.append(df)

In [9]:
df_expiry_hist_data

,strike_type,date,open,high,low,close,volume,oi
0,36000PE,2023-05-11 12:45:00+05:30,3.05,3.05,3.05,3.05,0.0,0.0
1,36000PE,2023-05-11 12:55:00+05:30,3.05,3.05,3.05,3.05,0.0,0.0
2,36000PE,2023-05-11 13:00:00+05:30,3.05,3.05,3.05,3.05,0.0,0.0
3,36000PE,2023-05-11 13:05:00+05:30,3.05,3.05,3.05,3.05,0.0,0.0
4,36000PE,2023-05-11 13:15:00+05:30,3.05,3.05,3.05,3.05,0.0,0.0
...,...,...,...,...,...,...,...,...
295,50000CE,2023-05-18 15:05:00+05:30,0.05,0.05,0.05,0.05,4650.0,583675.0
296,50000CE,2023-05-18 15:10:00+05:30,0.05,0.05,0.05,0.05,4300.0,583625.0
297,50000CE,2023-05-18 15:15:00+05:30,0.05,0.05,0.05,0.05,4175.0,583625.0
298,50000CE,2023-05-18 15:20:00+05:30,0.05,0.05,0.05,0.05,275.0,583300.0


In [10]:
df_expiry_hist_data.to_parquet('../data/expiry_18may_5min.parquet')